[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sujee/mongodb-atlas-vector-search/blob/main/lab-4-rag/rag-10k-b-query-mistral-embeddings-mistral-llm.ipynb)

#  RAG 10k Query: Mistral Embeddings with Mistral LLM

Here is the overall RAG pipeline.   In this notebook, we will do steps  (3), (4), (5), (6) and (7)
- Steps 1 & 2: populating embeddings.  It is already done in this notebook [rag-10k-a-populate-embeddings-mistral.ipynb](https://github.com/sujee/mongodb-atlas-vector-search/blob/main/lab-4-rag/rag-10k-a-populate-embeddings-mistral.ipynb)
- 👉 Step 3: Calculate embedding for user query
- 👉 Steps 4 & 5: Send the query to Atlas to retrieve relevant documents
- 👉 Steps 6 & 7 : Send the query and relevant documents (returned above step) to LLM and get answers to our query

![image missing](https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/images/rag-overview-3-mistral.png)

### What you need to run this notebook

- a (free) MongoDB Atlas Account
- and connection credentials
- a Mistral API Key

### This lab depends on:

- We assume we have processed PDF documents, calculated embeddings and loaded them into Atlas.  Refer to this notebook : [rag-10k-a-populate-embeddings-mistral.ipynb](https://github.com/sujee/mongodb-atlas-vector-search/blob/main/lab-4-rag/rag-10k-a-populate-embeddings-mistral.ipynb)

### The Stack

- Langugage : Python
- Vector database: Atlas
- Embedding Model: Mistral embedding model


### How to run

This notebook can be run on Google Colab and stand alone python development environments.  Click here to run on colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sujee/mongodb-atlas-vector-search/blob/main/lab-4-rag/rag-10k-b-query-mistral-embeddings-mistral-llm.ipynb)



## Step-1: Make sure documents is loaded in Atlas

This is done in this notebook: [rag-10k-a-populate-embeddings-mistral.ipynb](https://github.com/sujee/mongodb-atlas-vector-search/blob/main/lab-4-rag/rag-10k-a-populate-embeddings-mistral.ipynb)

Please complete this first.

## Step-2: Configuration

We will setup some common configurations here

In [1]:
# We will keep all global variables in an object to not pollute the global namespace.
class MyConfig(object):
    pass

MY_CONFIG = MyConfig()

MY_CONFIG.DB_NAME = 'rag1'
MY_CONFIG.COLLECTION_NAME = '10k_mistral'
MY_CONFIG.EMBEDDING_ATTRIBUTE = 'embedding_mistral'
MY_CONFIG.INDEX_NAME = 'idx_embedding_mistral'


## Step-3: Load Configuration

We need to configure the following
- Atlas connection credentials

### Option 3A - If running on Colab

- Click on 'Colab secrets' icon (🔑) on left pane, and crate the following secrets.
   - `ATLAS_URI`
   - `MISTRAL_API_KEY`
-  Make sure the `notebook access` button is checked on for all
- See screenshot below for example

<!-- ![](../images/colab-secret-2.png) -->

![](https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/images/colab-secret-3.png)


### Option 3B - If running on local python environment

- setup your local python env following this [setup guide](https://github.com/sujee/mongodb-atlas-vector-search/blob/main/setup-python-env.md)
- Create a file named `.env` in the same location as notebook
- And add the following settings

```text
ATLAS_URI=mongodb+srv://<username>:<password>@sandbox.....
MISTRAL_API_KEY=xyz
```


## Step-4: Determine Runtime Environment

This code will figure out if we are running on Google Colab environment or local environment.  We use it to install relevant packages later.

In [2]:
# are we running in Colab?
import os

if os.getenv("COLAB_RELEASE_TAG"):
    print("Running in Colab")
    MY_CONFIG.RUNNING_IN_COLAB = True
else:
    print("NOT running in Colab")
    MY_CONFIG.RUNNING_IN_COLAB = False

Running in Colab


## Step-5: Install dependencies (if necessary)

We will install required libraries in cloud environments like Google Colab.  For local environments, we assume the dependencies are already setup.

In [3]:
if MY_CONFIG.RUNNING_IN_COLAB:
    !pip install \
                pymongo==4.6.2 \
                llama-index \
                llama-index-embeddings-mistralai \
                llama-index-llms-mistralai \
                llama-index-vector-stores-mongodb \
                transformers==4.38.2 \
                torch==2.2.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.2/677.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━

## Step-6: Basic Setup

### 6.1 - Check if we have GPU

In [4]:
## Check if GPU is enabled
import os
import torch

## To disable GPU and experiment, uncomment the following line
## Normally, you would want to use GPU, if one is available.
# os.environ["CUDA_VISIBLE_DEVICES"]=""

print ("using CUDA/GPU: ", torch.cuda.is_available())

for i in range(torch.cuda.device_count()):
   print("device ", i , torch.cuda.get_device_properties(i).name)

using CUDA/GPU:  False


### 6.2 - Logging

In [5]:
## Setup logging.  To see more loging set the level to DEBUG

import sys
import logging

# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.basicConfig(stream=sys.stdout, level=logging.WARNING)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Step-7: Load Configurations

In [6]:
## Load settings based on where we are running
##  - if runninning on google Colab, load from secrets
##  - if running locally use dotenv

if MY_CONFIG.RUNNING_IN_COLAB:
    from google.colab import userdata
    MY_CONFIG.ATLAS_URI = userdata.get('ATLAS_URI')
    MY_CONFIG.MISTRAL_API_KEY = userdata.get('MISTRAL_API_KEY')
    # MY_CONFIG.OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
else:
    import os, sys
    from dotenv import find_dotenv, dotenv_values

    this_dir = os.path.abspath('')
    parent_dir = os.path.dirname(this_dir)
    sys.path.append (os.path.abspath (parent_dir))

    config = dotenv_values(find_dotenv())
    # debug
    # print (config)
    MY_CONFIG.ATLAS_URI = config.get('ATLAS_URI')
    MY_CONFIG.MISTRAL_API_KEY = config.get("MISTRAL_API_KEY")
## --- end load config

## If you just want to quickly set the config manually, you can do so here.
# MY_CONFIG.ATLAS_URI = ''
# MY_CONFIG.MISTRAL_API_KEY = ''

if  MY_CONFIG.ATLAS_URI:
    print ("✅ config ATLAS_URI found")
else:
    raise Exception ("'❌ ATLAS_URI' is not set.  Please set it above to continue...")


if MY_CONFIG.MISTRAL_API_KEY:
   print ("✅ config MISTRAL_API_KEY found")
else:
    raise Exception ("❌'MISTRAL_API_KEY' is not set.  Please set it above to continue...")

✅ config ATLAS_URI found
✅ config MISTRAL_API_KEY found


## Step-8: Initialize Atlas Client

If this step fails, make sure 'connect from anywhere' is enabled on your Atlas network configuration

![](https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/images/atlas-connect-2.png)

In [7]:
import pymongo

mongodb_client = pymongo.MongoClient(MY_CONFIG.ATLAS_URI)
print ('✅ Connected to Atlas instance!')

✅ Connected to Atlas instance!


## Step-9 : Setup Embeddings Model

We will use Mistral embedding

In [8]:
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.core import Settings

Settings.embed_model = MistralAIEmbedding(model_name='mistral-embed', api_key=MY_CONFIG.MISTRAL_API_KEY)

In [9]:
## testing
embeddings = Settings.embed_model.get_text_embedding("La Plateforme - The Platform")
print ('embedding len : ', len(embeddings))
print ('first few embeddings : ', embeddings[:10])

embedding len :  1024
first few embeddings :  [-0.0299224853515625, -0.0028362274169921875, 0.0282745361328125, -0.034759521484375, -0.0017366409301757812, 0.0112762451171875, 0.0022411346435546875, -0.018829345703125, -0.0160064697265625, -0.010955810546875]


## Step-10: Setup LLM

Our LLM of choice is Mistral

Here are the available models from https://docs.mistral.ai/guides/model-selection/

- open-mistral-7b (aka mistral-tiny-2312)
- open-mixtral-8x7b (aka mistral-small-2312)
- mistral-small-latest (aka mistral-small-2402)
- mistral-medium-latest (aka mistral-medium-2312)
- mistral-large-latest (aka mistral-large-2402)

In [10]:
from llama_index.llms.mistralai import MistralAI
from llama_index.core import Settings

llm = MistralAI(model="mistral-large-latest", temperature=0.1, api_key=MY_CONFIG.MISTRAL_API_KEY)

Settings.llm = llm

## Step-11: Setup Tokenizers

Setup tokenizers to match LLM for best results

In [11]:
from transformers import AutoTokenizer
from llama_index.core import Settings

tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2"
)

# set tokienizer for llama-index : https://docs.llamaindex.ai/en/stable/module_guides/supporting_modules/settings/#tokenizer
Settings.tokenzier = tokenizer #typo?
Settings.tokenizer = tokenizer


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [12]:
## test tokenizer
text = "Tokenizers are essential for natural language processing."
tokens = tokenizer.tokenize(text)
print ("Text words count : ", len (text.split()))
print ('tokens count: ', len(tokens))
print("Tokens:", tokens)

Text words count :  7
tokens count:  9
Tokens: ['▁Token', 'izers', '▁are', '▁essential', '▁for', '▁natural', '▁language', '▁processing', '.']


## Step-12 - Quick testing

In [13]:
## Testing
resp = llm.complete("The capital of the United States is ")
print (resp)

The capital of the United States is Washington, D.C. It was established in 1790 and is located on the east coast of the country, between the states of Maryland and Virginia. The city is named after George Washington, the first president of the United States, and the "D.C." stands for District of Columbia. Washington, D.C. is known for its iconic landmarks, such as the White House, the Capitol Building, and the Lincoln Memorial, as well as its many museums and cultural institutions.


## Step-13: Connect Illama-Index and MongoDB Atlas

Let's define MongoDB Atlas as our vector storage. This is critical to stored indexed data and then query

In [14]:
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex


vector_store = MongoDBAtlasVectorSearch(mongodb_client = mongodb_client,
                                        db_name = MY_CONFIG.DB_NAME,
                                        collection_name = MY_CONFIG.COLLECTION_NAME,
                                        index_name  = MY_CONFIG.INDEX_NAME,
                                        embedding_key = MY_CONFIG.EMBEDDING_ATTRIBUTE,
                                        ## the following columns are set to default values
                                       # text_key = 'text', metadata_= 'metadata',
                                 )
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_vector_store(vector_store=vector_store, storage_context=storage_context)

## Step-12: Query Data / Ask Questions

Now that we have every thing setup, let's ask some questions

These are the PDF documents we have loaded into Atlas, you can download them and inspect them.

- [10k/lyft_2021.pdf](https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/data/10k/lyft_2021.pdf)
- [10k/uber_2021.pdf](https://raw.githubusercontent.com/sujee/mongodb-atlas-vector-search/main/data/10k/uber_2021.pdf)

In [15]:
%%time

from IPython.display import Markdown
from pprint import pprint

response = index.as_query_engine().query("What was Uber's revenue?")
print (response)
print()
pprint(response, indent=4)

Uber's total revenue for the year ended December 31, 2021, was $17,455 million. This revenue is disaggregated into four main offerings: Mobility, Delivery, Freight, and All Other. The revenue from Mobility was $6,953 million, Delivery revenue was $8,362 million, Freight revenue was $2,132 million, and All Other revenue was $8 million for the same period.

Response(response="Uber's total revenue for the year ended December 31, 2021, "
                  'was $17,455 million. This revenue is disaggregated into '
                  'four main offerings: Mobility, Delivery, Freight, and All '
                  'Other. The revenue from Mobility was $6,953 million, '
                  'Delivery revenue was $8,362 million, Freight revenue was '
                  '$2,132 million, and All Other revenue was $8 million for '
                  'the same period.',
         source_nodes=[   NodeWithScore(node=TextNode(id_='58549811-0cef-4f75-bef6-e10c1a14a78d', embedding=None, metadata={'page_label': 

In [16]:
%%time

response = index.as_query_engine().query("How much money did Lyft make in 2020?")
print (response)
print()
pprint(response, indent=4)

In 2020, Lyft's revenue was $2,364,681 thousand. However, it's important to note that the company had a net loss of $1,752,857 thousand that year. This means that while they generated over $2 billion in revenue, their expenses exceeded their income, resulting in a net loss.

Response(response="In 2020, Lyft's revenue was $2,364,681 thousand. However, "
                  "it's important to note that the company had a net loss of "
                  '$1,752,857 thousand that year. This means that while they '
                  'generated over $2 billion in revenue, their expenses '
                  'exceeded their income, resulting in a net loss.',
         source_nodes=[   NodeWithScore(node=TextNode(id_='51ee5bff-1ac3-4688-b69c-5fa342ba7642', embedding=None, metadata={'page_label': '79', 'file_name': 'lyft_2021.pdf', 'file_path': '/content/data/10k/lyft_2021.pdf', 'file_type': 'application/pdf', 'file_size': 1440303, 'creation_date': '2024-03-22', 'last_modified_date': '2024-03-22'}, 

In [17]:
%%time

## The answer to this question doesn't exist in the Lyft_10k filing!
## Let's see what we get back
response = index.as_query_engine().query("How much money did Lyft make in 2018?")
print (response)
print()
pprint(response, indent=4)

The provided context does not include information about Lyft's revenue for the year 2018.

Response(response='The provided context does not include information about '
                  "Lyft's revenue for the year 2018.",
         source_nodes=[   NodeWithScore(node=TextNode(id_='596f6360-8592-4b1a-b3ce-b9d42f4c6545', embedding=None, metadata={'page_label': '58', 'file_name': 'lyft_2021.pdf', 'file_path': '/content/data/10k/lyft_2021.pdf', 'file_type': 'application/pdf', 'file_size': 1440303, 'creation_date': '2024-03-22', 'last_modified_date': '2024-03-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='176d2716-4db6-4091-877f-de73603ac9f7', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': 

In [18]:
%%time

response = index.as_query_engine().query("When did Uber go IPO?")
print (response)
print()
pprint(response, indent=4)

Uber went public on May 14, 2019.

Response(response='Uber went public on May 14, 2019.',
         source_nodes=[   NodeWithScore(node=TextNode(id_='a0f7944e-c16b-41c0-8654-6e6a27ed9fa8', embedding=None, metadata={'page_label': '119', 'file_name': 'uber_2021.pdf', 'file_path': '/content/data/10k/uber_2021.pdf', 'file_type': 'application/pdf', 'file_size': 1880483, 'creation_date': '2024-03-22', 'last_modified_date': '2024-03-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='69426eee-69ea-41f6-98a6-5581ca76e066', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '119', 'file_name': 'uber_2021.pdf', 'file_path': '/content/data/10k/uber_2021.pdf', 'file_type': 'application/pdf', 'file_size': 18

In [19]:
%%time

response = index.as_query_engine().query("What were the Stock-based compensation for Lyft?")
print (response)
print()
pprint(response, indent=4)

The stock-based compensation expense for Lyft was recorded on the consolidated statements of operations for the years ended December 31, 2021, 2020, and 2019. The total stock-based compensation expense for these years was $724,560, $565,807, and $1,599,311 respectively. These figures are distributed across different sectors such as cost of revenue, operations and support, research and development, sales and marketing, and general and administrative expenses.

In addition, in conjunction with one of the acquisitions in 2018, the company issued 241,390 shares of restricted stock awards to executives of an acquired company with an aggregate grant-date fair value of $11.4 million. These restricted stock awards are fully vested as of the year ended December 31, 2020. The company recorded $4.2 million and $6.0 million as compensation related to these vested restricted stock awards, which is included in research and development expense on the consolidated statement of operations for the years

## Try your own queries

Inspect the PDFs, and ask away.

Here are few things to try

- ask question about data in tables.  Are we getting accurate answers?
- Ask vague questions, e.g "which company is more environmentally friendly".  What do we get back?

In [20]:


%%time

# response = index.as_query_engine().query("YOUR QUERY GOES HERE")
# print (response)
# print()
# pprint(response, indent=4)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.39 µs
